# URL score calculator to determine if it is malicious or not.

## Models make use of Word2Vec (CBOW) and FastText for certain components

## imports

In [24]:
import pandas as pd
import keras
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model
from urllib.parse import urlparse,urlsplit
import gensim
from gensim.models import Word2Vec, FastText
from multiprocessing.pool import ThreadPool

## Prepare input data :

## split

In [2]:
def splitter(url_list):
    new_df=pd.DataFrame(columns=["scheme","netloc","path","query","fragment"])
    for url in url_list:
    #     test = [urlsplit(url).scheme, urlsplit(url).netloc, urlsplit(url).path, urlsplit(url).query, urlsplit(url).fragment]
        split_result=urlsplit(url)
        scheme = split_result.scheme
        netloc = split_result.netloc
        path = split_result.path
        query = split_result.query
        new_df =  new_df.append({'scheme': scheme, 'netloc': netloc, 'path': path, 'query': query}, ignore_index= True)
        new_df.fillna(0,inplace=True)
    return (new_df)

## vectorise using word2vec (cbow)

## fiddling

In [3]:
def vectorizer_netloc(new_df):
    
    model1 = Word2Vec.load("w2v_url")
    
    
    splitr1= new_df["netloc"]

    splitr1_1= [str(val).split(".") for val in splitr1.tolist()]
    
    model1.build_vocab(splitr1_1, update=True)
    model1.train(splitr1_1, total_examples=len(splitr1_1), epochs=1)

    
    holder1=[]
    vec_df1= pd.DataFrame()
    val=[0 for i in range(100)]
    for i in range(len(splitr1_1)):
        for j in range(len(splitr1_1[i])):
            val += model1[splitr1_1[i][j]]
        holder1.append(val.tolist())
        val *=0
    vec_df1=vec_df1.append(holder1)   
    
    
    return(vec_df1)

In [4]:
def vectorizer_path(new_df):
    
    model2 = Word2Vec.load("w2v_path")
    
    
    splitr2= new_df["path"]
    


    splitr2_1= [str(val).split("/") for val in splitr2.tolist()]
  

    model2.build_vocab(splitr2_1, update=True)
    model2.train(splitr2_1, total_examples=len(splitr2_1), epochs=1)
    

    holder2=[]
    vec_df2= pd.DataFrame()
    val=[0 for i in range(100)]
    for i in range(len(splitr2_1)):
        for j in range(len(splitr2_1[i])):
            val += model2[splitr2_1[i][j]]
        holder2.append(val.tolist())
        val *=0
    vec_df2=vec_df2.append(holder2) 
    
    
    return(vec_df2)

In [5]:
def vectorizer_query(new_df):
    
    model3 = Word2Vec.load("w2v_query")
    
    splitr3= new_df["query"]

    splitr3_1= [str(val).split("=") for val in splitr3.tolist()]


    model3.build_vocab(splitr3_1, update=True)
    model3.train(splitr3_1, total_examples=len(splitr3_1), epochs=1)

    holder3=[]
    vec_df3= pd.DataFrame()
    val=[0 for i in range(100)]
    for i in range(len(splitr3_1)):
        for j in range(len(splitr3_1[i])):
            val += model3[splitr3_1[i][j]]
        holder3.append(val.tolist())
        val *=0
    vec_df3=vec_df3.append(holder3) 
    
    
    return(vec_df3)

In [6]:
def convert_to_df(new_df):
    pool = ThreadPool(processes=2)
    new_df=new_df
    async_result1 = pool.apply_async(vectorizer_netloc, args=(new_df,))
    async_result2 = pool.apply_async(vectorizer_path, args=(new_df,))
    async_result3 = pool.apply_async(vectorizer_query, args=(new_df,))# tuple of args for foo



    
    vec_df1 = async_result1.get()
    vec_df2 = async_result2.get()
    vec_df3 = async_result3.get()
    
    tester_df_batch=pd.DataFrame()
    tester_df_batch=pd.concat([vec_df1,vec_df2,vec_df3], axis=1)
    return(tester_df_batch)

# URL score prediction, update, endpoint mischief scorer :

## url score updater

In [7]:
def return_avg_for_updated_url_score (eps,model):
    switch = []
    for key in eps.keys():
        for val in range(len(eps[key])):
            switch+=[(eps[key][val],key,calc_endpoint_score(model.predict(convert_to_df(splitter(eps[key]))).tolist()))]

    y =pd.DataFrame(switch, columns=['url','ip','eps_score'])

    grouped_sum = y.groupby('url').sum()
    grouped_size = y.groupby('url').size()
    # grouped_size['a']


    unique_urls = list(y['url'].unique())
    avg_scores_of_endpoints_hitting_the_same_url =[]
    for url in unique_urls :
        avg_scores_of_endpoints_hitting_the_same_url += [{url:grouped_sum['eps_score'][url]/grouped_size[url]}]

    return(avg_scores_of_endpoints_hitting_the_same_url,unique_urls)    

## endpoint score calculator

In [8]:
def calc_endpoint_score(score):
    day_score = float()
    bad_counter = int()
    for sc in score:

        if sc[0] > 0.5:
            day_score += sc[0]
            bad_counter += 1
        else:
            day_score += sc[0]

    bad_counter
    ep_w = (day_score/len(score))*(1.0-(1-(bad_counter/len(score))))
    return(ep_w)

## url name_vector relatedness 

In [9]:
def vector_relatedness(url):
    w_ = urlsplit(url)
    w_netloc = w_.netloc.split('.')
    w_path = w_.path.split('.')
    
    if(w_netloc[0]==''):

        related_word=w_path
        if(related_word[0] == 'www'):
            related_word_finder = related_word[1]

        else:
            related_word_finder = related_word[0]

    elif(w_netloc[0]=='www'):
        related_word_finder = w_netloc[1]

    else:
        related_word_finder = w_netloc[0]
        
        
    FastText_model = FastText.load('ft_.model')
    relatedness_vector = FastText_model.wv.most_similar_cosmul(related_word_finder)
    rv =np.mean([i[1] for i in relatedness_vector])
    return(rv)

## updating url scores based on relatedness and endpoint scores 

In [10]:
def url_score_recalculator(val_avg, unique_urls, url_scores):
    val_avg = val_avg
    unique_urls = unique_urls
    url_scores = [url_scores[i][0] for i in range(len(url_scores))]
    avg_list = [float(str(*list(val_avg[i].values()))) for i in range(len(val_avg))]
    
    rv =[vector_relatedness(url) for url in unique_urls]
    m_rv_url_score = np.add(np.multiply(url_scores,0.85),np.multiply(rv,0.1))
    
    m_avg_list = np.multiply(avg_list,0.05)
    
    updated_scores = np.subtract(m_rv_url_score,m_avg_list)
    updated_url_score_dict = [{unique_urls[i]:updated_scores[i]} for i in range(len(unique_urls))]
    return(updated_url_score_dict)

## inputs and outputs

In [11]:
eps = {'192.168.6.6.1':["https://www.yagle.com/abcd/?=123","https://www.google.com","www.bitsadmin.com/","https://www.yagle.com/abcd/?=123","https://www.google.com","www.bitsadmin.com/","http://zastapiony.piklamp.nl/military.xbel?face=jlthEvD&oil=70UfcEdppE&similar=0T9GXVd&plane=&never=XzUI&size=h-S&building=3NHL3LH4j&play=Ke9C4&over=&pattern=gKK","https://www.google.com"],
      '0.0.0.1':["https://www.yagle.com/abcd/?=123","https://www.google.com","www.bitsadmin.com/"]}

### on assuming an input of the above format to the code

model = load_model('current.best__std.hdf5')
val_avg, unique_urls = return_avg_for_updated_url_score(eps,model)

final_vec = convert_to_df(splitter(eps['192.168.6.6.1']))##
url_scores = model.predict(convert_to_df(splitter(unique_urls))).tolist()
updated_url_score_dict = url_score_recalculator(val_avg, unique_urls, url_scores)

score =  model.predict(final_vec).tolist()
ep_sc = calc_endpoint_score(score)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\adessai\anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\Users\adessai\anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\adessai\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\adessai\anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Wall time: 43.3 s


In [14]:
print("Endpoint Vectors for URLs of '192.168.6.6.1' (3 x 300) : ",final_vec)
print("\nScores for all unique URLs not bound by the endpoint: \n",url_scores)
print("\nUpdated URL scores to dict: \n",updated_url_score_dict)
print("\nScores for URLs of '192.168.6.6.1' endpoint: \n",score)
print("\nEndpoint ('192.168.6.6.1') score : ",ep_sc)

Endpoint Vectors for URLs of '192.168.6.6.1' (3 x 300) :           0         1         2         3         4         5         6   \
0  1.458456 -3.254446  1.979042 -0.257234 -2.897713  1.217711 -1.651971   
1  3.578903 -5.270251  2.364378  0.229436 -3.393633  1.455947 -2.137968   
2 -0.145892 -0.154081 -0.383800 -0.552012 -1.168146 -0.070009  0.452086   
3  1.458456 -3.254446  1.979042 -0.257234 -2.897713  1.217711 -1.651971   
4  3.578903 -5.270251  2.364378  0.229436 -3.393633  1.455947 -2.137968   
5 -0.145892 -0.154081 -0.383800 -0.552012 -1.168146 -0.070009  0.452086   
6  0.909108 -0.324216 -0.405326 -0.360043 -0.810911 -0.161119 -0.166233   
7  3.578903 -5.270251  2.364378  0.229436 -3.393633  1.455947 -2.137968   

         7         8         9   ...        90        91        92        93  \
0  0.164570 -1.647398 -1.130620  ...  0.199735 -0.440787 -0.280912  0.482175   
1  0.025913 -2.056639 -2.126576  ...  0.211939 -0.424193 -0.261291  0.470320   
2  0.525980  1.194449 -1.2

## future updates would make use of incremental learning structures